#### Importing relevant packages

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime
from bs4 import BeautifulSoup #requires pip install
import requests #requires pip install

In [2]:
#Test zoopla scraper
import pandas as pd
import numpy as np
#import matplotlib.pyplot as plt
import time
from bs4 import BeautifulSoup #requires pip install
import requests
import re
from re import sub
from decimal import Decimal
import io

# Convert price string into a numerical value
def to_num(price):
    value = Decimal(sub(r'[^\d.]', '', price))
    return float(value)

def is_dropped(money):
    for i in range(len(money)):
        if(money[i] != '£' and money[i] != ',' and (not money[i].isdigit())):
            return True
    return False


In [8]:
#set up the the scraper
url = 'https://www.zoopla.co.uk/for-sale/property/london/?page_size=25&q=london&radius=0&results_sort=newest_listings&pn='

map = {}
id = 0

#set max_pages to 2 for test purposes
max_pages = 3

# time.sleep(10)
start = time.time()

for p in range(max_pages):
    cur_url = url + str(p + 1)

    print("Scraping page: %d" % (p + 1))
    #print(cur_url)
    html_text = requests.get(cur_url).text
    soup = BeautifulSoup(html_text, 'lxml')

    ads = soup.find_all('div', class_ = 'css-wfndrn-StyledContent e2uk8e18')
    page_nav = soup.find_all('a', class_ = 'css-slm4qd-StyledPaginationLink eaoxhri5')

    if(len(page_nav) == 0):
        print("max page number: %d" % (p))
        # end = time.time()
        # print(end - start)
        break

    for k in range(len(ads)):
        ad = ads[k]

        #find link and ID ('identifier' in the link acts as a unique id for the ad)
        link = ad.find('a', class_ = 'e2uk8e4 css-gl9725-StyledLink-Link-FullCardLink e33dvwd0')

        #find section for address
        address = ad.find('p', class_ = 'css-wfe1rf-Text eczcs4p0').text

        #find price information
        price = ad.find('p', class_ = 'css-18tfumg-Text eczcs4p0').text

        # if the price is valid or not, if not we do not consider this ad
        if(is_dropped(price)): continue

        #find public transport information
        subway_section = ad.find('div', class_ = 'css-braguw-TransportWrapper e2uk8e28')
        subway_information = subway_section.find_all('p', class_ = 'css-wfe1rf-Text eczcs4p0')

        #skip ads that only contain information of train station
        outlier = subway_section.find('span', class_ = 'e1uy4ban0 css-10ibqwe-StyledIcon-Icon e15462ye0')
        if(outlier['data-testid'] == 'national_rail_station'): continue

        #find section for bedroom, bathroom and living room information (room numbers)
        feature_section = ad.find('div', class_ = 'css-58bgfg-WrapperFeatures e2uk8e15')

        #find all information available for room numbers
        category = feature_section.find_all('div', class_ = 'ejjz7ko0 css-l6ka86-Wrapper-IconAndText e3e3fzo1')

        #assign id
        ad_id = link['href'] #returns url snippet with identifier from the url
        ad_id= ad_id.split("?")[0] #split by '?' ans '/' and apply index to retain only identifier number
        ad_id= ad_id.split("/")[3]

        if(ad_id in map): continue
        map[ad_id] = {}

        #assign link
        link = 'https://www.zoopla.co.uk/' + link['href']
        map[ad_id]["link"] = link

        #assign address
        map[ad_id]["address"] = address

        #assign bedroom nr
        try:
            map[ad_id]["room_nr"] = category[0].text
        except IndexError:
        #insert None value if index is not found
            map[ad_id]["room_nr"] = 'None'
            #print("Feature not listed")

        #assign bathroom nr
        try:
            map[ad_id]["bath_nr"] = category[1].text
        except IndexError:
        #insert None value if index is not found
            map[ad_id]["bath_nr"] = 'None'
            #print("Feature not listed")

        #assign living room nr
        try:
            map[ad_id]["living_nr"] = category[2].text
        except IndexError:
        #insert None value if index is not found
            map[ad_id]["living_nr"] = 'None'
            #print("Feature not listed")

        #assign price
        map[ad_id]["price"] = to_num(price)

        #assign subway station and distance to it
        s = subway_information[0].text
        x = s.split(' miles ')
        if len(x) == 1: continue
        map[ad_id]["distance"] = float(x[0])
        map[ad_id]["subway_station"] = x[1]

print("Scraping task finished")

#transform to dict to list
result = []
cur_row = 0
for cur_id in map.keys():
    link = map[cur_id]["link"]
    cur_price = map[cur_id]["price"]
    cur_bedroom = map[cur_id]["room_nr"]
    cur_bathroom = map[cur_id]["bath_nr"]
    cur_living = map[cur_id]["living_nr"]
    cur_address = map[cur_id]["address"]
    cur_distance = map[cur_id]["distance"]
    cur_subway_station = map[cur_id]["subway_station"]
    result.append([])
    result[cur_row].append(str(cur_id))
    result[cur_row].append(str(link))
    result[cur_row].append(str(cur_price))
    result[cur_row].append(str(cur_bedroom))
    result[cur_row].append(str(cur_bathroom))
    result[cur_row].append(str(cur_living))
    result[cur_row].append(str(cur_address))
    result[cur_row].append(str(cur_distance))
    result[cur_row].append(str(cur_subway_station))
    cur_row += 1

#transform to dataframe
df = pd.DataFrame(result, columns = ["ad_id", "link", "price", "bedrooms", "bathrooms", "living_rooms", "address", "distance", "subway_station"])
df

Scraping page: 1
max page number: 0
Scraping task finished


,ad_id,link,price,bedrooms,bathrooms,living_rooms,address,distance,subway_station


In [10]:
#Dexter scraper:

# Import packages
import pandas as pd
import numpy as np
import datetime
from bs4 import BeautifulSoup #requires pip install
import requests #requires pip install
import re
import time

import io
# document time
time_started = str(datetime.datetime.now()).replace(" ","_").replace(":","-")[0:19]
## Define list of subway stations
Underground_lines = ['Bakerloo', 'Central', 'Circle', 'District', 'DLR', 'Hammersmith & City',
                 'Jubilee', 'Metropolitan', 'Northern', 'Piccadilly', 'Victoria', 'Waterloo & City']

## Function to extract characteristics on each ad from the main webpage
def feature_extract(html_text):

    soup = BeautifulSoup(html_text, 'lxml')

    ## Parse for the different divisions within the add

    # ads = soup.find_all('div', class_ = 'result-content') #searches for 'div' and is filtered by the CSS-snippet
    ads = soup.find_all('li', class_ = 'result item for-sale infinite-item')#searches for 'div' and is filtered by the CSS-snippet
    ## Set-up for the loop
    results = {} #create nested dictionary to store the results
    id_ad = 0 #insert ad_ID to distinguish between each ad

    ## Loop across all ads
    for k in range(len(ads)):
        ad = ads[k]
        id_ad += 1
        results[id_ad] = {}

        ## Extracting features from the ad
        name = ad.find('h3').a.contents[0]
        try:
            price = ad.find('span', class_ = 'price-qualifier').text #catches the price WITHIN one ad
        except:
            continue
        address = ad.find('span', class_ = 'address-area-post').text

        # Number of bedrooms extracted from string
        try:
            bedrooms = ad.find('li', class_ = 'Bedrooms').text
        except:
            continue
        bedrooms_nbr = int(bedrooms.split()[0])

        # Number of bedrooms extracted from string
        bathrooms_str = str(ad.find('li',class_ = 'Bathrooms'))
        bathrooms_nbr = re.findall(r'\d+', bathrooms_str)
        bathrooms_nbr2 = int(bathrooms_nbr[0] if len(bathrooms_nbr)!= 0  else 0)

        # Number of bedrooms extracted from string
        reception_str = str(ad.find('li',class_ = 'Receptions'))
        reception_nbr = re.findall(r'\d+', reception_str)
        reception_nbr2 = int(reception_nbr[0] if len(reception_nbr)!= 0  else 1)

        link = ad.find('h3').a.get("href")

        ad_identification = ads[k]['data-property-id']

        # Create dictionary of results per ad id
        results[id_ad]['ad_identification'] = ad_identification
        results[id_ad]["street_name"] = name
        results[id_ad]["price"] = price
        results[id_ad]["address"] = address
        results[id_ad]["bedrooms"] = bedrooms_nbr
        results[id_ad]["bathrooms"] = bathrooms_nbr2
        results[id_ad]["reception"] = reception_nbr2
        results[id_ad]["link"] = ("https://www.dexters.co.uk" + link)

        # Create dataframe from dictionary of results
        df_houses = pd.DataFrame.from_dict(results, orient='index')

    return df_houses

## Function to create list of pages base on url and number of iterations desired
def page_list(string, iterations):
    pages_list = []
    for i in range(iterations):
        pages_list.append(string + str(i+1))

    return pages_list

## Function to get the maximum number of listing on Dexter's website
def page_max(url):
    html_text = requests.get(url).text
    soup = BeautifulSoup(html_text, 'lxml')
    amount = soup.find('span', class_ = 'marker-count has-results').text
    amount_num = re.sub('\D', '', amount)
    return int(amount_num)

## Function to launch scrapper on a specific webpage with number of pages to scrap
def pages_scrap(main_page, iter_page, pages):
    max_pages = (page_max(main_page)/18)
    list_of_pages = page_list(iter_page, pages) # Create list of pages to scrape
    df_list = [] #Create list of dataframes to be concatenated by the end of the loop

    # Loop through all pages to create the different dataframes
    for page in list_of_pages:
        html_page = requests.get(page)
        html_page.encoding = 'utf-8'
        page = html_page.text
        df_ads = feature_extract(page)
        df_list.append(df_ads)

    # Concatenate the different dataframes
    df_results = pd.concat(df_list)
    df_results = df_results.drop_duplicates()
    df_results = df_results.reset_index(drop=True)

    print('Remaining number of page: ', int(max_pages - pages) )

    return df_results
# 1.2 Subway related functions

## Function to extract subway info list from a house webpage on dexter
def get_info_subway(link):
    html_text = requests.get(link).text
    soup = BeautifulSoup(html_text, 'lxml')
    subway = soup.find('ul', class_ = 'list-information').text

    return subway

## Function to get list of values for subway distances with string
def sub_values(string):
    split = string.split('\n')
    list_1 = list(filter(None, split))

    list_2 = []
    for i in list_1:
        x = i.split('-')
        list_2.append(x)

    list_3 = [item.strip() for sublist in list_2 for item in sublist]
    list_4 = list_3[0:3]

    return list_3

## Function to get the closest stop on the tube if any
def closest_line(list_of_lines):
    j = 0
    nearby_data = []
    for i in range(len(list_of_lines)):
        if list_of_lines[i] == 'London Underground' or list_of_lines[i] in Underground_lines and (j != 1 and i!=0):
            if (' ' in list_of_lines[i-2]) == False :
                nearby_data.append(list_of_lines[i-3])
                nearby_data.append(list_of_lines[i-2])
                nearby_data.append(list_of_lines[i-1])
                nearby_data.append(list_of_lines[i])
                j = 1

                nearby_data[0] = (' '.join(nearby_data[0:2]))
                del nearby_data[1]

            else:
                nearby_data.append(list_of_lines[i-2])
                nearby_data.append(list_of_lines[i-1])
                nearby_data.append(list_of_lines[i])
                j = 1

    return nearby_data

## Function to populate datafrmae with closest tube stop name, distance, and related tube line
def subway_per_house(df):
    #Create new empty (NaN) columns in the existing dataframe
    df = df.reindex(columns = df.columns.tolist() + ['subway_station','distance','tube_line'])

    #Loop through all lines of dataframe
    for i in range(len(df)):
        x = df['link'].iloc[i] #Get link of house page to scrape
        subs = get_info_subway(x) #Extract tube line info
        subs_2 = sub_values(subs) #Get list of subway station and distance
        subs_3 = closest_line(subs_2) #Extract closest tube station only

        # Populate dataframe if a tubeway station has been found or not
        if len(subs_3)!= 0:
            df['subway_station'].iloc[i] = subs_3[0]
            df['distance'].iloc[i] = subs_3[1]
            df['tube_line'].iloc[i] = subs_3[2]
        else:
            df['subway_station'].iloc[i] = np.NaN
            df['distance'].iloc[i] = np.NaN
            df['tube_line'].iloc[i] = np.NaN

    df = df.astype(str)

    return df

#Functions to clean subway output
def get_tube_dist(string):
    string_m = string.split(' ')
    num_val = string_m[-1]

    return num_val
def strip_tube(string):
    string_m = string.split(' ')
    string_m = string_m[:-1]
    string_m = ' '.join(string_m)

    return string_m
def hasNumbers(inputString):
    return any(char.isdigit() for char in inputString)

## Function to clean subway stops when too many words in the string
def clean_tube_stop_string(string):
    forbiddden_words = ['London Overground', 'Railway', 'Network Rail', 'Tramlink']
    count_forbidden = 0

    for j in forbiddden_words:
        if count_forbidden == 0:
            if j in string:
                string_update = string.split()[-1]
                count_forbidden = 1
            else:
                string_update = string

    return(string_update)

## Function to input tube distance into the right column when value is in 'tube_stop'
def clean_tube_dist(df):
    df['distance'] = df['distance'].astype('str')

    errors  = df[df.loc[:, 'distance'].map(hasNumbers) == False].copy()
    errors_2 = errors.loc[errors['subway_station'] != 'NaN'].copy()
    errors_2.loc[:, 'distance'] = errors_2.loc[:, 'subway_station'].map(get_tube_dist)
    errors_2.loc[:, 'subway_station'] = errors_2.loc[:, 'subway_station'].map(strip_tube)
    errors_2

    #Create copy of original df for modification
    df_copy = df.copy()

    # replace values in final df
    for i in errors_2.index:
        df_copy.loc[i] = errors_2.loc[i]

    return df_copy

## Functions to deal with Victoria tube stops (Victoria being both a tube stop and a tube line)
def victoria_clean_stop(string):
    str_vic = 'Victoria'
    str_check = string.split()
    if str_check[0] == 'Victoria':
        str_return = str_check[1]
    else:
        str_return = str_vic

    return str_return
def clean_tube_victoria(df):
    df['subway_station'] = df['subway_station'].astype('str')

    errors  = df[df['subway_station'].str.contains('Victoria')].copy()

    errors.loc[:, 'subway_station'] = errors.loc[:, 'subway_station'].map(victoria_clean_stop)

    #Create copy of original df for modification
    df_copy = df.copy()

    # Replace values in final df
    for i in errors.index:
        df_copy.loc[i] = errors.loc[i]

    return df_copy

## Final cleaning function to apply previous cleaning on 'tube_stop' and 'tube_dist' for the whole dataframe
def clean_tube_stop(df):
    df_2 = df.copy()
    df_2 = clean_tube_dist(df_2)
    df_2['subway_station'] = df_2['subway_station'].astype('str')
    df_2['subway_station'] = df_2['subway_station'].map(clean_tube_stop_string)

    df_2 = clean_tube_victoria(df_2)
    # #Keep the ID of the add as index or not


    return df_2

dexter_list_1 = pages_scrap('https://www.dexters.co.uk/property-sales/properties-for-sale-in-london',
                            'https://www.dexters.co.uk/property-sales/properties-for-sale-in-london/page-', 1)


## Fetch subway related information from the previous dataframe
output_list = subway_per_house(dexter_list_1)

output_list

cleaned = clean_tube_stop(output_list)
cleaned

Remaining number of page:  260


/Applications/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


,ad_identification,street_name,price,address,bedrooms,bathrooms,reception,link,subway_station,distance,tube_line
0,151907,Lancaster Gate,"£23,000,000","Hyde Park, W2",60,0,1,https://www.dexters.co.uk/property-for-sale/pr...,Lancaster Gate,0.26m,Central
1,124656,Wilton Crescent,"£19,500,000","Belgravia, SW1X",5,5,2,https://www.dexters.co.uk/property-for-sale/ho...,Knightsbridge,0.22m,Piccadilly
2,138546,Whitehall Place,"£9,250,000","Whitehall, SW1A",3,3,1,https://www.dexters.co.uk/property-for-sale/fl...,Charing Cross,0.12m,Bakerloo
3,143179,Old Queen Street,"£9,250,000","Westminster, SW1H",5,4,6,https://www.dexters.co.uk/property-for-sale/ho...,St.James's Park,0.16m,Circle
4,138545,Whitehall Place,"£9,000,000","Whitehall, SW1A",3,3,2,https://www.dexters.co.uk/property-for-sale/fl...,Charing Cross,0.12m,Bakerloo
5,138542,Whitehall Place,"£8,500,000","Whitehall, SW1A",2,2,2,https://www.dexters.co.uk/property-for-sale/fl...,Charing Cross,0.12m,Bakerloo
6,155928,Radnor Terrace,"£6,500,000","Kensington, W14",4,4,1,https://www.dexters.co.uk/property-for-sale/fl...,Kensington (Olympia),0.22m,District
7,138689,Queen Street,"£6,000,000","Mayfair, W1J",3,3,2,https://www.dexters.co.uk/property-for-sale/fl...,Green Park,0.18m,Jubilee
8,138690,Queen Street,"£6,000,000","Mayfair, W1J",3,3,1,https://www.dexters.co.uk/property-for-sale/fl...,East India,0.17m,DLR
9,145085,Hall Road,"£5,995,000","St John's Wood, NW8",5,5,5,https://www.dexters.co.uk/property-for-sale/pr...,Maida Vale,0.22m,Bakerloo


In [11]:
cleaned.shape

(15, 11)

In [3]:
epc_url='https://find-energy-certificate.digital.communities.gov.uk/find-a-certificate/search-by-street-name-and-town?street_name='
town_url='&town=London'
street_name='Bramwell Way'
url_street_name=street_name.replace(' ','+')
url_street_name
final_epc_url=epc_url+url_street_name+town_url
final_epc_url



'https://find-energy-certificate.digital.communities.gov.uk/find-a-certificate/search-by-street-name-and-town?street_name=Bramwell+Way&town=London'

In [4]:
html_text = requests.get(final_epc_url).text
soup = BeautifulSoup(html_text, 'lxml')

In [5]:
# To display full output in Notebook, instead of only the last result 
from IPython.core.interactiveshell import InteractiveShell 

InteractiveShell.ast_node_interactivity = "all" 

In [6]:
from statistics import mode


#for some reason it isn't 
epc_ads=soup.find_all('tr', class_ = 'govuk-table__row')
epc_ads[1]
# epc_ads

# results_epc = {}
epc_id_ad = 0
flat_address_final=[]
epc_rating=[]
for k in range(len(epc_ads)):
    ad = epc_ads[k]
#     results_epc[epc_id_ad] = {}
    

    
    flat_address = ad.find_all('a', class_='govuk-link')
#     flat_address = ad.find_all('a', href=True)
    
#     flat_address_final.append(flat_address[0].get_text(strip=True))
    if len(flat_address) !=0:
        flat_address_final.append(flat_address[0].get_text(strip=True))
    else: 
        continue
    
    epc_rating.append(ad.find_all('td', class_='govuk-table__cell')[0].get_text(strip=True))
       
    epc_id_ad += 1
#     flat_address = ad.find('a', class_='govuk-link')#.get_text()
#     flat_address = ad.get_text(strip=True)

epc_rating
len(flat_address_final)
len(epc_rating)
mode(epc_rating)
    
#     epc_rating=ad.find(class_='govuk-table__cell')
# flat_address

<tr class="govuk-table__row">
<th class="govuk-table__header" scope="row">
<a class="govuk-link" href="/energy-certificate/2338-5927-7320-3255-1910">
              Flat 3 Kingfisher Heights, 2, Bramwell Way, LONDON, E16 2GQ
            </a>
</th>
<td class="govuk-table__cell">B</td>
<td class="govuk-table__cell">
<span style="white-space: nowrap;">
              26 October 2025
            </span>
</td>
</tr>

['B',
 'B',
 'B',
 'B',
 'C',
 'C',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'C',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'C',
 'C',
 'C',
 'C',
 'C',
 'C',
 'C',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'C',
 'B',
 'B',
 'C',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'C',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'C',
 'C',
 'C',
 'B',
 'B',
 'B',
 'B',
 'B',
 'C',
 'C',
 'B',
 'B',
 'B',
 'B',
 'C',
 'B',
 'B',
 'B',
 'B',
 'C',
 'C',
 'B',
 'B',
 'B',
 'C'

194

194

'B'

In [139]:
#get the results of EPC ratings for all the street names in the Dexter scraper:

#first create a list of URLs for scraping
list_epc_urls=[]
epc_url='https://find-energy-certificate.digital.communities.gov.uk/find-a-certificate/search-by-street-name-and-town?street_name='
town_url='&town=London'
epc_avg=[]

for street_name in list(cleaned['street_name']):
    url_street_name=street_name.replace(' ','+')
    list_epc_urls.append(epc_url+url_street_name+town_url)

    #iterate through
for url in list_epc_urls:
    
    html_text = requests.get(url).text
    soup = BeautifulSoup(html_text, 'lxml')
    epc_ads=soup.find_all('tr', class_ = 'govuk-table__row')
    
    epc_id_ad = 0
    flat_address_final=[]
    epc_rating=[]
    for k in range(len(epc_ads)):
        ad = epc_ads[k]
        flat_address = ad.find_all('a', class_='govuk-link')
  
        if len(flat_address) !=0:
            flat_address_final.append(flat_address[0].get_text(strip=True))
        else: 
            continue

        epc_rating.append(ad.find_all('td', class_='govuk-table__cell')[0].get_text(strip=True))

        epc_id_ad += 1
    epc_avg.append(mode(epc_rating))
avg_epc_tuple=list(zip(list(cleaned['street_name']),epc_avg))
street_name_epc=pd.DataFrame(avg_epc_tuple, columns=['street_name','average_epc'])
street_name_epc

,street_name,average_epc
0,Lancaster Gate,C
1,Wilton Crescent,D
2,Whitehall Place,D
3,Old Queen Street,C
4,Whitehall Place,D
5,Whitehall Place,D
6,Radnor Terrace,B
7,Queen Street,C
8,Queen Street,C
9,Hall Road,D


In [7]:
#trying to get the full list of London streets 
#link: http://london.streetmapof.co.uk/


london_streetmap='http://london.streetmapof.co.uk/'
alphabet_letter='a'
streetmap_url=london_streetmap+alphabet_letter+'/'

    
map_html_text = requests.get(streetmap_url).text
soup_map = BeautifulSoup(map_html_text, 'lxml')

#find street names in the map:
map_ads=soup_map.find_all('td', { "valign": "top" })


# ad = map_ads[1]
count_pages=[]
[count_pages.append(a) for a in soup_map.find_all('a', href=True) if a['href'].startswith(f'/{alphabet_letter}/')]
len(count_pages) 
count_pages

    

[None, None, None, None, None, None]

6

[<a href="/a/" style="color:#000">[ 1 ]</a>,
 <a href="/a/2" style="color:#AAA">[ 2 ]</a>,
 <a href="/a/3" style="color:#AAA">[ 3 ]</a>,
 <a href="/a/4" style="color:#AAA">[ 4 ]</a>,
 <a href="/a/5" style="color:#AAA">[ 5 ]</a>,
 <a href="/a/6" style="color:#AAA">[ 6 ]</a>]

In [8]:
#FINAL for the bigger for loop to collect all the elements of the list for letter "A"
list_starting_withA=[]
for i in range(len(count_pages)):
    if i==0:
        streetmap_url=london_streetmap+alphabet_letter+'/'
        print("page",i," ",streetmap_url)
        map_html_text = requests.get(streetmap_url).text
        soup_map = BeautifulSoup(map_html_text, 'lxml')
        map_ads=soup_map.find_all('td', { "valign": "top" })
        for k in range(len(map_ads)):
            ad = map_ads[k]


            for a in ad.find_all('a', href=True):
                if a['href'].startswith(f'/{alphabet_letter}/'):
                    count_pages.append(a)
                else:
                    list_starting_withA.append(a.get_text(strip=True))
    else:
        streetmap_url=london_streetmap+alphabet_letter+'/'+str(i+1)
        print("page",i," ",streetmap_url)
        map_html_text = requests.get(streetmap_url).text
        soup_map = BeautifulSoup(map_html_text, 'lxml')
        map_ads=soup_map.find_all('td', { "valign": "top" })

        for k in range(len(map_ads)):
            ad = map_ads[k]


            for a in ad.find_all('a', href=True):
                if a['href'].startswith(f'/{alphabet_letter}/'):
                    count_pages.append(a)
                else:
                    list_starting_withA.append(a.get_text(strip=True))
list_starting_withA
len(set(list_starting_withA))

page 0   http://london.streetmapof.co.uk/a/
page 1   http://london.streetmapof.co.uk/a/2
page 2   http://london.streetmapof.co.uk/a/3
page 3   http://london.streetmapof.co.uk/a/4
page 4   http://london.streetmapof.co.uk/a/5
page 5   http://london.streetmapof.co.uk/a/6


['Aaron Hill Road',
 'Abberley Mews',
 'Abbess Close',
 'Abbeville Road',
 'Abbey Drive',
 'Abbey Gardens',
 'Abbey Grove',
 'Abbey Lane',
 'Abbey Mews',
 'Abbey Orchard Street',
 'Abbey Parade',
 'Abbey Road',
 'Abbey Street',
 'Abbey Terrace',
 'Abbey View',
 'Abbey Wood Road',
 'Abbeyfield Estate',
 'Abbeyfield Road',
 'Abbeyfields Close',
 'Abbots Gardens',
 'Abbots Manor',
 'Abbots Park',
 'Abbots Place',
 'Abbots Road',
 'Abbots Terrace',
 'Abbots Walk',
 'Abbotsbury Close',
 'Abbotsbury Mews',
 'Abbotsbury Road',
 'Abbotsford Avenue',
 'Abbotshade Road',
 'Abbotshall Avenue',
 'Abbotshall Road',
 'Abbotsleigh Road',
 'Abbotstone Road',
 'Abbotswell Road',
 'Abbotswood Road',
 'Abbott Avenue',
 'Abbott Road',
 'Abbotts Close',
 'Abbotts Crescent',
 'Abbotts Park Road',
 'Abchurch Lane',
 'Abdale Road',
 'Aberavon Road',
 'Abercairn Road',
 'Abercorn Close',
 'Abercorn Place',
 'Abercorn Road',
 'Abercorn Way',
 'Abercrombie Street',
 'Aberdare Gardens',
 'Aberdeen Lane',
 'Aberde

1159

In [9]:
list_starting_withA
len(set(list_starting_withA))

['Aaron Hill Road',
 'Abberley Mews',
 'Abbess Close',
 'Abbeville Road',
 'Abbey Drive',
 'Abbey Gardens',
 'Abbey Grove',
 'Abbey Lane',
 'Abbey Mews',
 'Abbey Orchard Street',
 'Abbey Parade',
 'Abbey Road',
 'Abbey Street',
 'Abbey Terrace',
 'Abbey View',
 'Abbey Wood Road',
 'Abbeyfield Estate',
 'Abbeyfield Road',
 'Abbeyfields Close',
 'Abbots Gardens',
 'Abbots Manor',
 'Abbots Park',
 'Abbots Place',
 'Abbots Road',
 'Abbots Terrace',
 'Abbots Walk',
 'Abbotsbury Close',
 'Abbotsbury Mews',
 'Abbotsbury Road',
 'Abbotsford Avenue',
 'Abbotshade Road',
 'Abbotshall Avenue',
 'Abbotshall Road',
 'Abbotsleigh Road',
 'Abbotstone Road',
 'Abbotswell Road',
 'Abbotswood Road',
 'Abbott Avenue',
 'Abbott Road',
 'Abbotts Close',
 'Abbotts Crescent',
 'Abbotts Park Road',
 'Abchurch Lane',
 'Abdale Road',
 'Aberavon Road',
 'Abercairn Road',
 'Abercorn Close',
 'Abercorn Place',
 'Abercorn Road',
 'Abercorn Way',
 'Abercrombie Street',
 'Aberdare Gardens',
 'Aberdeen Lane',
 'Aberde

1159

In [10]:
#import string for alphabet
import string
from statistics import mode
import itertools
flatten = itertools.chain.from_iterable

In [11]:
%%time
#attempting to scrape the streets for the entire alphabet
list_london_street_names=[]
for letter in list(string.ascii_lowercase):

    london_streetmap='http://london.streetmapof.co.uk/'
    streetmap_url=london_streetmap+letter+'/'


    map_html_text = requests.get(streetmap_url).text
    soup_map = BeautifulSoup(map_html_text, 'lxml')

    #find street names in the map:
    map_ads=soup_map.find_all('td', { "valign": "top" })


    # ad = map_ads[1]
    count_pages=[]
    [count_pages.append(a) for a in soup_map.find_all('a', href=True) if a['href'].startswith(f'/{letter}/')]
    len(count_pages)

    list_starting_with_letter=[]
    if len(count_pages)>0:
        list_starting_with_letter=[]
        for i in range(len(count_pages)):
            if i==0:
                streetmap_url=london_streetmap+letter+'/'

                map_html_text = requests.get(streetmap_url).text
                soup_map = BeautifulSoup(map_html_text, 'lxml')
                map_ads=soup_map.find_all('td', { "valign": "top" })
                for k in range(len(map_ads)):
                    ad = map_ads[k]


                    for a in ad.find_all('a', href=True):
                        if a['href'].startswith(f'/{letter}/'):
                            count_pages.append(a)
                        else:
                            list_starting_with_letter.append(a.get_text(strip=True))
            else:
                streetmap_url=london_streetmap+letter+'/'+str(i+1)

                map_html_text = requests.get(streetmap_url).text
                soup_map = BeautifulSoup(map_html_text, 'lxml')
                map_ads=soup_map.find_all('td', { "valign": "top" })

                for k in range(len(map_ads)):
                    ad = map_ads[k]


                    for a in ad.find_all('a', href=True):
                        if a['href'].startswith(f'/{letter}/'):
                            count_pages.append(a)
                        else:
                            list_starting_with_letter.append(a.get_text(strip=True))
        len(list(set(list_starting_with_letter)))
        list_london_street_names.append(list(set(list_starting_with_letter)))
    else:
        list_starting_with_letter=[]
        streetmap_url=london_streetmap+letter+'/'

        map_html_text = requests.get(streetmap_url).text
        soup_map = BeautifulSoup(map_html_text, 'lxml')
        map_ads=soup_map.find_all('td', { "valign": "top" })
        for k in range(len(map_ads)):
            ad = map_ads[k]


            for a in ad.find_all('a', href=True):
                if a['href'].startswith(f'/{letter}/'):
                    break
                else:
                    list_starting_with_letter.append(a.get_text(strip=True))
        len(list(set(list_starting_with_letter)))
        list_london_street_names.append(list(set(list_starting_with_letter)))
        
list_london_street_names


CPU times: user 8.5 s, sys: 196 ms, total: 8.69 s
Wall time: 25.6 s


[['Acris Street',
  'Avarn Road',
  'Arborfield Close',
  'All Saints Close',
  'Aldrington Road',
  'Arndale Walk',
  'Augustus Road',
  'Ambler Road',
  'Adair Road',
  'Aldwick Close',
  'Albert Carr Gardens',
  'Archibald Road',
  'Arctic Street',
  'Ashlar Place',
  'Aston Road',
  'Atheldene Road',
  'Akenside Road',
  'Andrews Walk',
  'Acre Lane',
  'Aaron Hill Road',
  'Adys Lawn',
  'Agnes Street',
  'Archdale Road',
  'Aslett Street',
  'Ashingdon Close',
  'Armstrong Close',
  'Arran Drive',
  'Alkerden Road',
  'Amyruth Road',
  'Acland Road',
  'Ashurst Gardens',
  'Alexander Mews',
  'Ackroyd Drive',
  'Ancona Road',
  'Alt Grove',
  'Arden Crescent',
  'Anderson Close',
  'Acacia Road',
  'Admirals Walk',
  'Argyll Street',
  'Abbeville Road',
  'Adelaide Road',
  'Achilles Road',
  'Alderton Crescent',
  'Allhallows Road',
  'Ann Moss Way',
  'Arica Road',
  'Amblecote Close',
  'Apple Tree Yard',
  'Arkell Grove',
  'Anderton Close',
  'Amherst Avenue',
  'Ash Road',


In [12]:
#flatten the list of lists for london street names
flatten = itertools.chain.from_iterable
list_london_street_names=list(flatten(list_london_street_names))
len(list_london_street_names)

23337

In [11]:
#FINAL for the bigger for loop to collect all the elements of the list for letter "Z" because it's a short list
alphabet_letter='z'
list_starting_withZ=[]
for i in range(len(count_pages)):
    if i==0:
        streetmap_url=london_streetmap+alphabet_letter+'/'
        print("page",i," ",streetmap_url)
        map_html_text = requests.get(streetmap_url).text
        soup_map = BeautifulSoup(map_html_text, 'lxml')
        map_ads=soup_map.find_all('td', { "valign": "top" })
        for k in range(len(map_ads)):
            ad = map_ads[k]


            for a in ad.find_all('a', href=True):
                if a['href'].startswith(f'/{alphabet_letter}/'):
                    count_pages.append(a)
                else:
                    list_starting_withZ.append(a.get_text(strip=True))
    else:
        streetmap_url=london_streetmap+alphabet_letter+'/'+str(i+1)
        print("page",i," ",streetmap_url)
        map_html_text = requests.get(streetmap_url).text
        soup_map = BeautifulSoup(map_html_text, 'lxml')
        map_ads=soup_map.find_all('td', { "valign": "top" })

        for k in range(len(map_ads)):
            ad = map_ads[k]


            for a in ad.find_all('a', href=True):
                if a['href'].startswith(f'/{alphabet_letter}/'):
                    count_pages.append(a)
                else:
                    list_starting_withZ.append(a.get_text(strip=True))
list_starting_withZ=set(list_starting_withZ)
len(set(list_starting_withZ))

page 0   http://london.streetmapof.co.uk/z/
page 1   http://london.streetmapof.co.uk/z/2
page 2   http://london.streetmapof.co.uk/z/3
page 3   http://london.streetmapof.co.uk/z/4
page 4   http://london.streetmapof.co.uk/z/5
page 5   http://london.streetmapof.co.uk/z/6
page 6   http://london.streetmapof.co.uk/z/7
page 7   http://london.streetmapof.co.uk/z/8
page 8   http://london.streetmapof.co.uk/z/9
page 9   http://london.streetmapof.co.uk/z/10
page 10   http://london.streetmapof.co.uk/z/11
page 11   http://london.streetmapof.co.uk/z/12
page 12   http://london.streetmapof.co.uk/z/13
page 13   http://london.streetmapof.co.uk/z/14
page 14   http://london.streetmapof.co.uk/z/15
page 15   http://london.streetmapof.co.uk/z/16
page 16   http://london.streetmapof.co.uk/z/17
page 17   http://london.streetmapof.co.uk/z/18
page 18   http://london.streetmapof.co.uk/z/19
page 19   http://london.streetmapof.co.uk/z/20
page 20   http://london.streetmapof.co.uk/z/21
page 21   http://london.streetmap

8

In [13]:
list_starting_withZ

NameError: name 'list_starting_withZ' is not defined

In [15]:
%%time
#get the results of EPC ratings for all the street names in London starting with Z:
#first create a list of URLs for scraping
list_epc_urls=[]
epc_url='https://find-energy-certificate.digital.communities.gov.uk/find-a-certificate/search-by-street-name-and-town?street_name='
town_url='&town=London'
epc_avg=[]

for street_name in list_starting_withZ:
    url_street_name=street_name.replace(' ','+')
    list_epc_urls.append(epc_url+url_street_name+town_url)

    #iterate through
for url in list_epc_urls:
    
    html_text = requests.get(url).text
    soup = BeautifulSoup(html_text, 'lxml')
    epc_ads=soup.find_all('tr', class_ = 'govuk-table__row')
    
    epc_id_ad = 0
    flat_address_final=[]
    epc_rating=[]
    for k in range(len(epc_ads)):
        ad = epc_ads[k]
        flat_address = ad.find_all('a', class_='govuk-link')
  
        if len(flat_address) !=0:
            flat_address_final.append(flat_address[0].get_text(strip=True))
        else: 
            continue

        epc_rating.append(ad.find_all('td', class_='govuk-table__cell')[0].get_text(strip=True))

        epc_id_ad += 1
    if len(epc_rating)!=0:
        epc_avg.append(mode(epc_rating))
    else:
        epc_avg.append(0)
avg_epc_tuple=list(zip(list_starting_withZ,epc_avg))
street_name_epc=pd.DataFrame(avg_epc_tuple, columns=['street_name','average_epc'])
street_name_epc

CPU times: user 411 ms, sys: 22.9 ms, total: 434 ms
Wall time: 1min 1s


,street_name,average_epc
0,Zenoria Street,D
1,Zander Court,C
2,Zennor Road,0
3,Zangwill Road,D
4,Zoffany Street,B
5,Zoar Street,0
6,Zealand Road,D
7,Zetland Street,D


In [ ]:
%%time
#get the results of EPC ratings for all the street names in London FINAL:
#first create a list of URLs for scraping
list_epc_urls=[]
epc_url='https://find-energy-certificate.digital.communities.gov.uk/find-a-certificate/search-by-street-name-and-town?street_name='
town_url='&town=London'
epc_avg=[]

for street_name in list_london_street_names:
    url_street_name=street_name.replace(' ','+')
    list_epc_urls.append(epc_url+url_street_name+town_url)

    #iterate through
for url in list_epc_urls:
    
    html_text = requests.get(url).text
    soup = BeautifulSoup(html_text, 'lxml')
    epc_ads=soup.find_all('tr', class_ = 'govuk-table__row')
    
    epc_id_ad = 0
    flat_address_final=[]
    epc_rating=[]
    for k in range(len(epc_ads)):
        ad = epc_ads[k]
        flat_address = ad.find_all('a', class_='govuk-link')
  
        if len(flat_address) !=0:
            flat_address_final.append(flat_address[0].get_text(strip=True))
        else: 
            continue

        epc_rating.append(ad.find_all('td', class_='govuk-table__cell')[0].get_text(strip=True))

        epc_id_ad += 1
    if len(epc_rating)!=0:
        epc_avg.append(mode(epc_rating))
    else:
        epc_avg.append(0)
avg_epc_tuple=list(zip(list_london_street_names,epc_avg))
street_name_epc=pd.DataFrame(avg_epc_tuple, columns=['street_name','average_epc'])
street_name_epc